# Tarea 4
En esta tarea usted deberá completar el siguiente notebook, en el cual va a implementar el protocolo de firmas de anillo sobre grupos en general. Debe completar exclusivamente las partes marcadas con `##### POR COMPLETAR`.

## Funciones auxiliares
Necesitaremos un test de primalidad para revisar que no nos estén engañando cuando nos entreguen los parámetros del grupo.

In [2]:
import random

def _is_natural_power(n):
    # Para cada posible exponente, hacemos búsqueda binaria de la base
    search_exponent = 2
    
    # Optimiazación: si n no es a ^ k no puede ser a ^ (kr) para ningún
    # r, por lo que sólo probamos con exponentes primos
    avoid_exponents = set()
    
    while pow(2, search_exponent) <= n:
        
        if search_exponent not in avoid_exponents:
            # Usamos búsqueda binaria "creciente" para definir el intervalo
            # inicial
            search_start = 2
            i = 2
            while search_start ** search_exponent < n:
                search_start *= 2
                avoid_exponents.add(search_exponent * i)
                i += 1
                
            upper = search_start
            lower = search_start // 2

            # Búsqueda binaria
            while lower != upper:
                mid = (upper + lower) // 2
                result = pow(mid, search_exponent)
                if result < n:
                    lower = mid + 1
                elif result > n:
                    upper = mid
                else:
                    return True

            # Caso borde en que upper ^ search_exponent era justo n
            if pow(upper, search_exponent) == n:
                return True
            
        search_exponent += 1
    
    return False

In [3]:
def _extended_euclid(a, b):
    if a > b:
        return _extended_euclid_base(a, b)
    return _extended_euclid_base(b, a)

def _extended_euclid_base(a, b):
    prev_r, r = a, b
    prev_s, s = 1, 0
    prev_t, t = 0, 1

    while r != 0:
        q = prev_r // r
        prev_r, r = r, prev_r % r
        prev_s, s = s, prev_s - q * s
        prev_t, t = t, prev_t - q * t

    return prev_r, prev_s, prev_t

In [4]:
def is_probably_prime(n, iterations=100):
    if n == 2:
        return True
    if n % 2 == 0 or n == 1:
        return False
    if _is_natural_power(n):
        return False
    
    found_negative = False
    for i in range(iterations):
        a = random.randint(1, n - 1)
        if _extended_euclid(a, n)[0] > 1:
            return False
        b = pow(a, (n - 1) // 2, n)
        if b == n - 1:
            found_negative = True
        elif b != 1:
            return False
    
    return found_negative

## Un primer grupo
Como un primer ejemplo de grupo, consideramos a los grupos Z<sub>p</sub><sup>\*</sup> vistos en clases. En particular, definimos la clase `ZpStar` cuyas instancias son estos grupos. Para representar a los elementos dentro de Z<sub>p</sub><sup>\*</sup>, en su constructor se crea dinámicamente la clase `Element`.

In [5]:
class ZpStar:
    
    def __init__(self, p):
        if not is_probably_prime(p):
            raise Exception(f"p={p} is not a prime number")
        class Element:
            def __init__(self, value):
                if value < 1 or value > p-1:
                    raise Exception(f"value={value} is not in the range 1,...,{p-1}")
                self.value = value

            # Allows to compare elements with ==
            def __eq__(self, other_element):
                return self.value == other_element.value

            # Allows to operate elements with *
            def __mul__(self, other_element):
                return Element((self.value * other_element.value) % p)

            # Allows to use ** as exponentiation
            def __pow__(self, exponent):
                return Element(pow(self.value, exponent, p))

            # Allows to use str(e) to transform an element into a string
            def __str__(self):
                return str(self.value)

        self.element_class = Element
                
    def get_identity(self):
        return self.get_element(1)
    
    def get_element(self, n):
        return self.element_class(n)

Ahora vamos a definir una clase que inicializa a los participantes del anillo. Cada participante es una instancia de la clase `Signer` que será definida más adelante. De la misma forma que para la clase `ZpStar`, su implementación del constructor de la clase `RingSetup` debe generar excepciones si los parámetros entregados no son correctos (puede suponer que los tipos de estos parámetros siempre van a ser los correctos). Por ejemplo, si `subgroup_order` no es un número primo, entonces se debe generar una excepción (puede suponer que el valor entregado en `subgroup_order` va a ser un número natural mayor o igual a 1).

In [6]:
import random

class RingSetup:
    def __init__(self, generator, subgroup_order, n_participants, group):
        # Is the order of the generator correct? For this we check that
        # 1. The subgroup order is prime
        # 2. The generator to the power of subgroup_order is the identity
        # 3. The generator is not the identity
        ##### POR COMPLETAR
        if not is_probably_prime(subgroup_order):
            raise Exception("Subroup order is not prime, it's a trap!")
        if generator == group.get_identity():
            raise Exception("The generator is the identity, it's a trap!")
        if generator ** subgroup_order != group.get_identity():
            raise Exception("This is not the real order, it's a trap!")
    
        # Generate a group of participants
        self.participants = [
            Signer(generator, subgroup_order) for _ in range(n_participants)
        ]

        # Store their public keys
        self.public_keys = [x.get_public_key() for x in self.participants]

    def get_public_keys(self):
        return self.public_keys

    def get_random_participant(self):
        return random.choice(self.participants)

Ahora definimos la clase `Signer`.

In [7]:
from hashlib import sha256

class Signer():
    def __init__(self, generator, subgroup_order, secret_key, public_key):
        self.generator = generator
        self.subgroup_order = subgroup_order
        self.secret_key = secret_key
        self.public_key = public_key

    def get_public_key(self):
        return self.public_key

    # Compute a ring signature for a message and a list of public keys
    def generate_ring_signature(self, message, public_keys):
        # Simplify notation
        q = self.subgroup_order
        g = self.generator
        n = len(public_keys)

        my_r = random.randint(1, q - 1)
        print(f"my r {my_r}")
        my_index = public_keys.index(self.public_key)

        signatures = [0] * len(public_keys)
        challenges = [0] * len(public_keys)
        challenges[(my_index + 1) % n] = int.from_bytes(sha256((str(g ** my_r) + message).encode()).digest()) % q

        for i in range(1, n):
            index = (my_index + i) % n
            signatures[index] = random.randint(1, q - 1)
            R = g ** signatures[index] * public_keys[index] ** (q - challenges[(index)])
            challenges[(index + 1) % n] = int.from_bytes(sha256((str(R) + message).encode()).digest()) % q

        signatures[my_index] = (my_r + challenges[my_index] * self.secret_key) % q
        print(f"s_{my_index}: {signatures[my_index]}")

        random_index = random.randint(0, n - 1)
        
        return signatures, challenges[random_index], random_index

In [8]:
class Verifier:
    def __init__(self, generator, subgroup_order):
        self.generator = generator
        self.subgroup_order = subgroup_order

    def verify_ring_signature(self, public_keys, message, signatures, challenge, challenge_index):
        # Verify a ring signature
        # simplify notation
        q = self.subgroup_order
        g = self.generator
        n = len(public_keys)
        current_challenge = challenge

        for i in range(1, n + 1):
            index = (challenge_index + i) % n
            prev = (index - 1) % n
            R = g ** signatures[prev] * public_keys[prev] ** (q - current_challenge) 
            current_challenge = int.from_bytes(sha256((str(R) + message).encode()).digest()) % q


        return challenge == current_challenge

In [19]:
if __name__ == "__main__":
    p = 17125458317614137930196041979257577826408832324037508573393292981642667139747621778802438775238728592968344613589379932348475613503476932163166973813218698343816463289144185362912602522540494983090531497232965829536524507269848825658311420299335922295709743267508322525966773950394919257576842038771632742044142471053509850123605883815857162666917775193496157372656195558305727009891276006514000409365877218171388319923896309377791762590614311849642961380224851940460421710449368927252974870395873936387909672274883295377481008150475878590270591798350563488168080923804611822387520198054002990623911454389104774092183
    generator = 8041367327046189302693984665026706374844608289874374425728797669509435881459140662650215832833471328470334064628508692231999401840332046192569287351991689963279656892562484773278584208040987631569628520464069532361274047374444344996651832979378318849943741662110395995778429270819222431610927356005913836932462099770076239554042855287138026806960470277326229482818003962004453764400995790974042663675692120758726145869061236443893509136147942414445551848162391468541444355707785697825741856849161233887307017428371823608125699892904960841221593344499088996021883972185241854777608212592397013510086894908468466292313
    order = 63762351364972653564641699529205510489263266834182771617563631363277932854227

    group = ZpStar(p)
    g = group.get_element(generator)

    
    p0 = group.get_element(3023321843356641609024850187414150159940016123551412410956004775306007902209445074828510213524699300006518257045640971265884379649843078545839146013271177397854289310617244525030328818870605803495015296951026319700358217061371468803722294922510119740393445753660621444186639220804875431477820678065035574583985316778773739946335798885478136935033628369393364265223680328934665562780177568106290187090673859662611448678539857231494322154358023615796395892769458852147373472495589387082021243210756747854766726006554775319151353965543740685166644601351191797766603106670695258564926670603645225992140308137705033126071)
    p1 = group.get_element(8672791744732106257696875242982937110288561412496602335410715863953744781693471649791448077284516383422850715536069265929500839549215939248764280245994145641975711320668871500305371166238843624591247878219996488751317419623558448680521304816807891773165691942540722787654377294579107889894284544979770483744445900229097591113386046934305938806324320443111671166581839057656982148084822748796947396126796217827130707284138366784451795320589327950829276191382665845774227480749384259469644124268871844432411985630722250124694920370161280576455377746614606658607260827970287637921831720467730481605710271073095225740930)
    s2 = 22378796404047169028538460741634530254779024799681816607876638960720903797322
    p2 = group.get_element(451021976850407374189567941850253632832341858059333460507329186590160411002200724775462299270663349073503028155952999262245547696667463103512291305242989212244654744832059829307256488654088727798799984062395882279725695733455785612762340460098902293909288452525222917393787515205286253725415895459822581687336870503266359615990943848396702701110926673330882823891515134181306090859074644386354656283735128016198692630243944395958714611280645475903316955705310562351027632522484236560041565456362325799470809520033853227483508038483257654782221740733957273386416481011739200540155352165580829227942419604433072927255)
    p3 = group.get_element(1905870283543360857514965720297067393598468377530585869933588910917452387264450049223285552883053378010124792516898501166646024979806825126107612153256752532902906863217283089620083771357114359571006971774638699679043117407921880767054901775311688958497395480345353681070839587433336861913314346171687640810242444757125685328732189768198075806215610130076136025110485789859471116071285891833475348160812543921734323239336679699481119369117538656859047737087820927492610042333104169034702782539670248545945306530002202420079616488816269226329217715435554837972569259560026021594613045696772243511957950923662834825391)
    ps = [p0, p1, p2, p3]

    
    signer = Signer(g, order, s2, p2)

    
    message = "Si alguien descubre quien firmo este mensaje tiene un 7 final en el curso, en serio!"
    print(f"Message:             {message}")

    print(signer.get_public_key())
    signatures, challenge, challenge_index = signer.generate_ring_signature(message, ps)

    print(f"{signatures[0]},{signatures[1]},{signatures[2]},{signatures[3]},{challenge},{challenge_index}")
    
    verifier = Verifier(g, order)
    result = verifier.verify_ring_signature(ps, message, signatures, challenge, challenge_index)
    print(f"Correct signature:   {result}")

    signatures[0] = (signatures[0] - 1) % order
    result = verifier.verify_ring_signature(ps, message, signatures, challenge, challenge_index)
    print(f"Incorrect signature: {result}")

Message:             Si alguien descubre quien firmo este mensaje tiene un 7 final en el curso, en serio!
451021976850407374189567941850253632832341858059333460507329186590160411002200724775462299270663349073503028155952999262245547696667463103512291305242989212244654744832059829307256488654088727798799984062395882279725695733455785612762340460098902293909288452525222917393787515205286253725415895459822581687336870503266359615990943848396702701110926673330882823891515134181306090859074644386354656283735128016198692630243944395958714611280645475903316955705310562351027632522484236560041565456362325799470809520033853227483508038483257654782221740733957273386416481011739200540155352165580829227942419604433072927255
my r 33872398055935947259725237309151987046562817587844289744814643484146586404110
s_2: 54960665162849582844359114361107496473622288104074422163196737077268139173040
37025695368633273351697133098655483698625234967861682960248535277794672549155,1842812240740395573952957275954243

In [ ]:
p0 = 3023321843356641609024850187414150159940016123551412410956004775306007902209445074828510213524699300006518257045640971265884379649843078545839146013271177397854289310617244525030328818870605803495015296951026319700358217061371468803722294922510119740393445753660621444186639220804875431477820678065035574583985316778773739946335798885478136935033628369393364265223680328934665562780177568106290187090673859662611448678539857231494322154358023615796395892769458852147373472495589387082021243210756747854766726006554775319151353965543740685166644601351191797766603106670695258564926670603645225992140308137705033126071
p1 = 8672791744732106257696875242982937110288561412496602335410715863953744781693471649791448077284516383422850715536069265929500839549215939248764280245994145641975711320668871500305371166238843624591247878219996488751317419623558448680521304816807891773165691942540722787654377294579107889894284544979770483744445900229097591113386046934305938806324320443111671166581839057656982148084822748796947396126796217827130707284138366784451795320589327950829276191382665845774227480749384259469644124268871844432411985630722250124694920370161280576455377746614606658607260827970287637921831720467730481605710271073095225740930
s2 = 22378796404047169028538460741634530254779024799681816607876638960720903797322
p2 = 451021976850407374189567941850253632832341858059333460507329186590160411002200724775462299270663349073503028155952999262245547696667463103512291305242989212244654744832059829307256488654088727798799984062395882279725695733455785612762340460098902293909288452525222917393787515205286253725415895459822581687336870503266359615990943848396702701110926673330882823891515134181306090859074644386354656283735128016198692630243944395958714611280645475903316955705310562351027632522484236560041565456362325799470809520033853227483508038483257654782221740733957273386416481011739200540155352165580829227942419604433072927255
p3 = 1905870283543360857514965720297067393598468377530585869933588910917452387264450049223285552883053378010124792516898501166646024979806825126107612153256752532902906863217283089620083771357114359571006971774638699679043117407921880767054901775311688958497395480345353681070839587433336861913314346171687640810242444757125685328732189768198075806215610130076136025110485789859471116071285891833475348160812543921734323239336679699481119369117538656859047737087820927492610042333104169034702782539670248545945306530002202420079616488816269226329217715435554837972569259560026021594613045696772243511957950923662834825391

In [ ]:
23130175073952556754837949876845820197049404380063539064453637992889583004254
30446881779762775041349758758331369600764995183746592443873942047921812862627
18916198306688353205824508700768735417381900159223013133982233692968254730442
6541689303016971068849098764635247893948787040887572916511675324319824710555
26919301156190709827156662787092856776498896743640633326610249827890755631379
1

In [13]:
52432898422521481681577426566927387630693273319562719668097951177255597574580
45649543664860668268566458792958116727367744683321906546531126820777514993095
54687556489214799228196750695867788821458430170257679021468454252258799531275
52432898422521481681577426566927387630693273319562719668097951177255597574580
33459699874035079242558807440412591889512889221698067918770290213523799953325
22883181989394515251846417569176707189734623345137906909300553536973145988271
3



3